In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
my_url = 'http://www.soccer-db.info/index.php?option=com_php&views=php&Itemid=224&team1=0&team2=0&tour=all&limitstart=0'
#opening the connection and grabbing the content
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

In [3]:
page_soup = soup(page_html, "html.parser")

In [12]:
table = page_soup.findAll('table')[2]
table

<table border="0" cellpadding="3" cellspacing="2" width="100%">
<tr style="background-color:#DADEE0; text-align:center;">
<td>Date</td>
<td>Venue</td>
<td>Match</td>
<td>Score</td>
<td>Tournament</td>
<td>Season</td>
<td>Link</td>
</tr>
<tr style="background-color:#ffffff;">
<td style="text-align:center;">2018-06-18</td>
<td style="text-align:center;">Nizhny Novgorod, Russia</td>
<td style="text-align:center;">
<a href="/index.php?option=com_joomsport&amp;task=team&amp;tid=29&amp;sid=818" target="_blank">Sweden</a> - <a href="/index.php?option=com_joomsport&amp;task=team&amp;tid=31&amp;sid=818" target="_blank">Korea Republic</a>
</td>
<td style="text-align:center;">1:0</td>
<td style="text-align:center;">FIFA World Cup</td>
<td style="text-align:center;"><a href="/index.php?option=com_joomsport&amp;view=ltable&amp;sid=818&amp;gr_id=0" target="_blank">2018</a></td>
<td style="text-align:center;"><a href="/index.php?option=com_joomsport&amp;task=view_match&amp;id=46061" target="_blank">D

In [67]:
import pandas as pd
import numpy as np
new_table = pd.DataFrame(columns=range(0,7), index=np.arange(51))

In [68]:
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        new_table.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    if row_marker == 0:
        new_table.columns = new_table.loc[0,:]
    row_marker += 1
new_table.drop([0], inplace =True)
new_table.head()

,Date,Venue,Match,Score,Tournament,Season,Link
1,2018-06-18,"Nizhny Novgorod, Russia",\nSweden - Korea Republic\n,1:0,FIFA World Cup,2018,Details»
2,2018-06-18,"Sochi, Russia",\nBelgium - Panama\n,3:0,FIFA World Cup,2018,Details»
3,2018-06-17,"Rostov-on-Don, Russia",\nBrazil - Switzerland\n,1:1,FIFA World Cup,2018,Details»
4,2018-06-17,"Moscow, Russia",\nGermany - Mexico\n,0:1,FIFA World Cup,2018,Details»
5,2018-06-17,"Samara, Russia",\nCosta Rica - Serbia\n,0:1,FIFA World Cup,2018,Details»


In [69]:
new_table.drop(['Season', 'Link'], axis = 1, inplace=True)
new_table.head()

,Date,Venue,Match,Score,Tournament
1,2018-06-18,"Nizhny Novgorod, Russia",\nSweden - Korea Republic\n,1:0,FIFA World Cup
2,2018-06-18,"Sochi, Russia",\nBelgium - Panama\n,3:0,FIFA World Cup
3,2018-06-17,"Rostov-on-Don, Russia",\nBrazil - Switzerland\n,1:1,FIFA World Cup
4,2018-06-17,"Moscow, Russia",\nGermany - Mexico\n,0:1,FIFA World Cup
5,2018-06-17,"Samara, Russia",\nCosta Rica - Serbia\n,0:1,FIFA World Cup


In [70]:
new_table['Home'] = new_table['Match'].apply(lambda x: x.split(' - ')[0][1:])
new_table['Away'] = new_table['Match'].apply(lambda x: x.split(' - ')[1][:-1])
new_table.drop(['Match'], axis = 1, inplace = True)
new_table['Venue'] = new_table['Venue'].apply(lambda x: x.split(', ')[1])
new_table['Home_score'] = new_table['Score'].apply(lambda x: int(x.split(':')[0]))
new_table['Away_score'] = new_table['Score'].apply(lambda x: int(x.split(':')[1]))
new_table.drop(['Score'], axis = 1, inplace = True)
new_table['is_win'] = new_table['Home_score'] - new_table['Away_score']

def win_draw_lose(x):
    if x > 0:
        return 'Win'
    elif x < 0:
        return 'Lose'
    else:
        return 'Draw'

new_table['is_win'] = new_table['is_win'].apply(win_draw_lose)
new_table.head()

,Date,Venue,Tournament,Home,Away,Home_score,Away_score,is_win
1,2018-06-18,Russia,FIFA World Cup,Sweden,Korea Republic,1,0,Win
2,2018-06-18,Russia,FIFA World Cup,Belgium,Panama,3,0,Win
3,2018-06-17,Russia,FIFA World Cup,Brazil,Switzerland,1,1,Draw
4,2018-06-17,Russia,FIFA World Cup,Germany,Mexico,0,1,Lose
5,2018-06-17,Russia,FIFA World Cup,Costa Rica,Serbia,0,1,Lose


In [72]:
new_table['Date'] = pd.to_datetime(new_table['Date'])

In [74]:
new_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 8 columns):
Date          50 non-null datetime64[ns]
Venue         50 non-null object
Tournament    50 non-null object
Home          50 non-null object
Away          50 non-null object
Home_score    50 non-null int64
Away_score    50 non-null int64
is_win        50 non-null object
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 3.5+ KB
